1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
- 토큰수 초과로 답변을 생성하지 못할 수 있고
- 문서가 길면 (인풋이 길면) 답변 생성이 오래거림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
%pip install --upgrade --quiet docx2txt langchain-community
%pip install -qU langchain-text-splitters

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1500,
  chunk_overlap=200, # 문서를 겹치게 하면 유사도 검색을 할 때 우리가 원하는 문서를 가지고 올 확률을 높임
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter)


In [3]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv() # OpenAIEmbeddings의 로드를 위해서 필요

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
%pip install langchain-chroma

In [5]:
from langchain_chroma import Chroma

database = Chroma.from_documents(
  documents=document_list,
  embedding=embeddings,
  collection_name="tax_collection",
  persist_directory="./chroma_db"
)
# database = Chroma(
#   collection_name="tax_collection",
#   persist_directory="./chroma_db",
#   embedding_function=embeddings
# )


In [6]:
query = "연봉 1억인 직장인의 소득세는 얼마인가요?" # 존댓말로 하면 답변이 더 좋을 수 있다? 라는 썰이 있다고 한다.
# retrieved_docs = database.similarity_search(query, k=3) # 내 쿼리에 맞춰 유사도 검색을 하는데 관련 문서(데이터)를 3개를 가져온다.

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [ ]:
# 페르소나를 주면 좋다?
# 프롬프트도 별도 작성할 필요 없이 langchain hub에 있는 프롬프트를 사용하면 된다.
# prompt = f"""[Identity]
# - 당신은 최고의 한국 소득세 전문가 입니다.
# - [Context]를 참고해서 사용자의 질문에 답변해주세요

# [Context]
# {retrieved_docs}

# [Question]
# {query}
# """

# ai_message = llm.invoke(prompt)

# ai_message.content

In [ ]:
%pip install -U langchain langchainhub --quiet

In [ ]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [10]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
  llm=llm,
  retriever=database.as_retriever(),
  chain_type_kwargs={"prompt": prompt}
)

In [ ]:
ai_message = qa_chain({"query": query})
ai_message